In [1]:
# !pip install Praw
# !pip install --upgrade praw

In [2]:
import sys
!{sys.executable} -m pip install praw
!{sys.executable} -m pip install psaw
import praw
import pandas as pd
import datetime as dt
import json

f = open("../secret_python.txt", "r")
client_id = f.readline().rstrip()
secret_key = f.readline().rstrip()
f.close()

## Praw

In [3]:
reddit = praw.Reddit(client_id=client_id, \
                     client_secret=secret_key, \
                     user_agent='Scrape')

reddit.read_only

True

In [4]:
dank_meme = reddit.subreddit('dank_meme')

In [5]:
memesDict = {}
memesDict["_default"] = {}

Source: https://stackoverflow.com/questions/40960449/getting-all-submissions-for-the-past-two-months-from-a-particular-subreddit-usi

In [6]:
iterator = 1
for submission in dank_meme.hot(limit = 100):
    #print(dt.datetime.fromtimestamp(submission.created_utc)) 
    memesDict["_default"][iterator] = {"title":submission.title, "thumbnail":{"thumbnail":submission.thumbnail, "height":submission.thumbnail_height, "width":submission.thumbnail_width}, "created_utc":submission.created_utc, "author":submission.author.name, "id":submission.id, "ups":submission.ups, "downs":submission.downs, "media": submission.url}
    iterator +=1

In [7]:
print(memesDict["_default"][2])

{'title': 'do not remove', 'thumbnail': {'thumbnail': 'https://b.thumbs.redditmedia.com/heg3pmxBhdqySBbmLvBQcv2XqTExKt7mP2tkft-3DHM.jpg', 'height': 140, 'width': 140}, 'created_utc': 1587025856.0, 'author': 'IKnowTheyAreReal', 'id': 'g2b27q', 'ups': 324, 'downs': 0, 'media': 'https://i.redd.it/9dg2e4jf45t41.png'}


In [8]:
with open('memesDatabase.json', 'w') as json_file:
    json.dump(memesDict, json_file)

In [9]:
#Used to find out what fields does an object have
import pprint
#pprint.pprint(vars(submission))

## Pushshift

With help from the following link: https://medium.com/@RareLoot/using-pushshifts-api-to-extract-reddit-submissions-fb517b286563

In [10]:
import requests

In [11]:
base_url = "https://api.pushshift.io/"
fields="author,subreddit,id,created_utc,all_awardings,score,thumbnail,thumbnail_height,thumbnail_width,title,url,full_link"
def queryBySub(sub, before="30d", after="60d", sort_type="score", sort="asc",size="25", fields=fields):
    url = base_url + "reddit/search/submission/?subreddit="+sub+"&sort="+sort+"&sort_type="+sort_type+"&size="+size+"&after="+after+"&before="+before+"&fields="+fields
    response = requests.get(url)
    return json.loads(response.text)
data = queryBySub("dank_meme")

In [12]:
data["data"]

[{'all_awardings': [],
  'author': 'Popcorny_YT',
  'created_utc': 1583363896,
  'full_link': 'https://www.reddit.com/r/dank_meme/comments/fdlrju/hmmmm_yes_gold_robbery/',
  'id': 'fdlrju',
  'score': 0,
  'subreddit': 'dank_meme',
  'thumbnail': 'https://b.thumbs.redditmedia.com/5OkmOvdbo7IC86JCv8gkwH6EkBIuvgMnypM0LCua_2c.jpg',
  'thumbnail_height': 140,
  'thumbnail_width': 140,
  'title': 'Hmmmm yes gold robbery',
  'url': 'https://i.redd.it/5vldqep9oqk41.jpg'},
 {'all_awardings': [],
  'author': 'elisportsfan',
  'created_utc': 1584394640,
  'full_link': 'https://www.reddit.com/r/dank_meme/comments/fjsxxo/me_at_walmart_yesterday/',
  'id': 'fjsxxo',
  'score': 0,
  'subreddit': 'dank_meme',
  'thumbnail': 'https://b.thumbs.redditmedia.com/6uvcxxHPlAdrTz3WBNL2nJUFMSO81H_1LPVhqGOGuzQ.jpg',
  'thumbnail_height': 73,
  'thumbnail_width': 140,
  'title': 'Me At Walmart Yesterday',
  'url': 'https://i.redd.it/0i6yrb77t3n41.jpg'},
 {'all_awardings': [],
  'author': 'The_choochebandit_69',

## Python Pushshift.io API Wrapper 

API: https://github.com/dmarx/psaw

In [13]:
import psaw
from psaw import PushshiftAPI

api = PushshiftAPI(reddit)

In [31]:
current_time = dt.datetime.now()
end_date = current_time-dt.timedelta(days = 30) #how many days before today you want the search to end
start_date = current_time-dt.timedelta(days = 34) #how many days before today do you want the search to start
start_epoch = int(dt.datetime.timestamp(start_date))
end_epoch = int(dt.datetime.timestamp(end_date))

In [15]:
subr = 'dank_meme'
searchResults = api.search_submissions(after=start_epoch,
                            before=end_epoch,
                            subreddit=subr)
cache = []

for result in searchResults:
    cache.append(result)
    
cache[0:10]

[Submission(id='fk6615'),
 Submission(id='fk63a0'),
 Submission(id='fk610x'),
 Submission(id='fk5ryu'),
 Submission(id='fk5ob0'),
 Submission(id='fk5nt6'),
 Submission(id='fk5mvl'),
 Submission(id='fk5hzj'),
 Submission(id='fk5a72'),
 Submission(id='fk5812')]

In [16]:
pprint.pprint(vars(cache[1]))

{'_comments_by_id': {},
 '_fetched': False,
 '_reddit': <praw.reddit.Reddit object at 0x000002B704267248>,
 'all_awardings': [],
 'allow_live_comments': False,
 'approved_at_utc': None,
 'approved_by': None,
 'archived': False,
 'author': Redditor(name='Brinks0088'),
 'author_flair_background_color': None,
 'author_flair_css_class': None,
 'author_flair_richtext': [],
 'author_flair_template_id': None,
 'author_flair_text': None,
 'author_flair_text_color': None,
 'author_flair_type': 'text',
 'author_fullname': 't2_xolhb3z',
 'author_patreon_flair': False,
 'author_premium': False,
 'awarders': [],
 'banned_at_utc': None,
 'banned_by': None,
 'can_gild': False,
 'can_mod_post': False,
 'category': None,
 'clicked': False,
 'comment_limit': 2048,
 'comment_sort': 'best',
 'content_categories': None,
 'contest_mode': False,
 'created': 1584483819.0,
 'created_utc': 1584455019.0,
 'discussion_type': None,
 'distinguished': None,
 'domain': 'i.redd.it',
 'downs': 0,
 'edited': False,
 'gi

In [17]:
memesDict2 = {}
memesDict2["_default"] = {}

In [18]:
iterator = 1
maxUpvote = 0
for submission in cache:
    if (submission is not None and submission.is_video is False): #ignore video memes
        #print(dt.datetime.fromtimestamp(submission.created_utc)) 
        if (submission.ups > maxUpvote):
            maxUpvote = submission.ups
        memesDict2["_default"][iterator] = {"title":submission.title, "thumbnail":{"thumbnail":submission.thumbnail, "height":submission.thumbnail_height, "width":submission.thumbnail_width}, "created_utc":submission.created_utc, "id":submission.id, "ups":submission.ups, "downs":submission.downs, "media": submission.url, "url": submission.permalink, "awards": submission.all_awardings, 'subreddit':submission.subreddit.display_name}
        if (submission.author is not None): #in case the account is deleted
            memesDict2["_default"][iterator]["author"] = submission.author.name 
        else:
            memesDict2["_default"][iterator]["author"] = "N/A"
        iterator +=1

In [19]:
memesDict2['_default'][2]

{'title': "I've got a 93 Toyota Paseo out back.... Let's get outta here.",
 'thumbnail': {'thumbnail': 'https://b.thumbs.redditmedia.com/lQRSWEKcOlNMZhz7gtN8Hq6Hc4aBppGNCHv0DFmxyro.jpg',
  'height': 108,
  'width': 140},
 'created_utc': 1584455019.0,
 'id': 'fk63a0',
 'ups': 11,
 'downs': 0,
 'media': 'https://i.redd.it/ngg8wmpqs8n41.jpg',
 'url': '/r/dank_meme/comments/fk63a0/ive_got_a_93_toyota_paseo_out_back_lets_get_outta/',
 'awards': [],
 'subreddit': 'dank_meme',
 'author': 'Brinks0088'}

There are some thumbnails with value as 'default' btw.

In [20]:
subr = 'meme'
searchResults = api.search_submissions(after=start_epoch,
                            before=end_epoch,
                            subreddit=subr)
cache = []

for result in searchResults:
    cache.append(result)
    
cache[0:10]

[Submission(id='fk66mr'),
 Submission(id='fk66d9'),
 Submission(id='fk664g'),
 Submission(id='fk663k'),
 Submission(id='fk65us'),
 Submission(id='fk65u1'),
 Submission(id='fk65ry'),
 Submission(id='fk654j'),
 Submission(id='fk64em'),
 Submission(id='fk644e')]

In [21]:
for submission in cache:
    if (submission is not None and submission.is_video is False):
        #print(dt.datetime.fromtimestamp(submission.created_utc)) 
        if (submission.ups > maxUpvote):
            maxUpvote = submission.ups
        memesDict2["_default"][iterator] = {"title":submission.title, "thumbnail":{"thumbnail":submission.thumbnail, "height":submission.thumbnail_height, "width":submission.thumbnail_width}, "created_utc":submission.created_utc, "id":submission.id, "ups":submission.ups, "downs":submission.downs, "media": submission.url, "url": submission.permalink, "awards": submission.all_awardings}
        if (submission.author is not None):
            memesDict2["_default"][iterator]["author"] = submission.author.name 
        else:
            memesDict2["_default"][iterator]["author"] = "N/A"
        iterator +=1

In [22]:
maxUpvote

26323

In [23]:
len(memesDict2['_default'])

15414

In [25]:
with open('memesDatabase2.json', 'w') as json_file:
    json.dump(memesDict2, json_file)

In [32]:
import sys
# !{sys.executable} -m pip install opencv-python
import cv2
from skimage import io
def url_to_cvimage(url):
    try:
        image = io.imread(url)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        cv2.waitKey(0)
        return image.tolist()
    except:
        return "N/A"
    
def cache_search_results_by_sub(subr):
    api = PushshiftAPI(reddit)
    searchResults = api.search_submissions(after=start_epoch,
                            before=end_epoch,
                            subreddit=subr)
    return list(searchResults)

In [33]:
import requests
def isBrokenURL(url):
    if "youtu" in url:
        return True
    try:
        if requests.get(url).status_code == 404:
            return True
    except:
        return True
    return False

In [34]:
def make_dict_from_cache(cache):
    maxUpvote = 0
    iterator = 1
    memesDict = {}
    memesDict["_default"] = {}
    for submission in cache:
        if (submission is not None and submission.is_video is False):
            if (isBrokenURL(submission.url)): #no videos or no broken links
                if (submission.ups > maxUpvote):
                    maxUpvote = submission.ups
                memesDict["_default"][iterator] = {"title":submission.title, "thumbnail":{"thumbnail":submission.thumbnail, "height":submission.thumbnail_height, "width":submission.thumbnail_width}, "created_utc":submission.created_utc, "id":submission.id, "ups":submission.ups, "downs":submission.downs, "media": submission.url, "url": submission.permalink, "awards": submission.all_awardings, "subreddit":submission.subreddit_name_prefixed, "subscribers": submission.subreddit_subscribers, "is_nsfw": submission.over_18}
                if (submission.author is not None):
                    memesDict["_default"][iterator]["author"] = submission.author.name 
                else:
                    memesDict["_default"][iterator]["author"] = "N/A"
                iterator +=1
    return memesDict

def make_json_from_dict(meme_dict, sub_name):
    with open('./Data/memesDatabase_' + sub_name + '.json', 'w') as json_file:
        json.dump(meme_dict, json_file)
        
def add_cv(meme_dict):
    for i in meme_dict["_default"]:
        meme_dict["_default"][i]["cv_image"] = url_to_cvimage(meme_dict["_default"][i]["media"])

In [35]:
subreddits = ["MemeEconomy", "memes", "me_irl", "dank_meme", "dankmeme"]

In [36]:
meme_dicts = []
for sub in subreddits:
    meme_dicts.append(make_dict_from_cache(cache_search_results_by_sub(sub)))

ConnectionError: HTTPSConnectionPool(host='www.change.org', port=443): Max retries exceeded with url: /p/the-rt-hon-gavin-williamson-cbe-mp-covid-19-award-predicted-grades-to-a-level-students?utm_content=cl_sharecopy_20879854_en-GB%3Av3&recruited_by_id=29a99970-6829-11ea-a657-3392bd11d0f3&utm_source=share_petition&utm_medium=copylink&utm_campaign=psf_combo_share_initial&utm_term=psf_combo_share_initial (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x000002B7738229C8>: Failed to establish a new connection: [Errno 11002] getaddrinfo failed'))

In [ ]:
for i in range(len(meme_dicts)):
    make_json_from_dict(meme_dicts[i], subreddits[i])